In [2]:
from requests import get
url = 'https://www.imdb.com/list/ls000729643/'
response = get(url)

In [3]:
from bs4 import BeautifulSoup
raw_html = BeautifulSoup(response.text, 'html.parser')
movies = raw_html.find_all('div', class_ = 'lister-item mode-detail')

In [18]:
# Lists to store the scraped data in
ranks = []
titles = []
descriptions = []
runtimes = []
genres = []
ratings = []
metascores = []
votes = []
gross_earnings = []
directors = []
actors = []


# Extract data from individual movie container
for movie in movies:

        # The rank
        rank = movie.h3.find('span', class_ = 'text-primary').text 
        ranks.append(rank)
        
        # The name
        title = movie.h3.a.text
        titles.append(title)
        
        # The description
        raw_description = movie.find('p', class_ = '').text
        description = raw_description.replace("\n","")
        descriptions.append(description)
        
        # The runtime
        raw_runtime = movie.p.find('span', class_ = 'runtime').text
        runtime = raw_runtime.replace("min","")
        runtimes.append(int(runtime))
        
        # The genre
        raw_genre = movie.p.find('span', class_ = 'genre').text
        genre = raw_genre.replace("\n","")
        genres.append(genre)
        
        # The IMDB rating
        rating = float(movie.find('span', class_='ipl-rating-star__rating').text)
        ratings.append(rating)

        # The Metascore
        metascore = movie.find('span', class_ = 'metascore').text
        metascores.append(int(metascore))

        # The number of votes
        vote = movie.find('span', attrs = {'name':'nv'})['data-value']
        votes.append(int(vote))
        
        # The gross earning in millions
        raw_earning = movie.select("span[name=nv]")[1].text
        raw_earning_wo_dollar = raw_earning.replace("$","")
        earning = raw_earning_wo_dollar.replace("M","")
        gross_earnings.append(float(earning)) 
        
        # List directors and actors
        raw_directorandactor = movie.select("p.text-muted.text-small")[1].text
        directorandactor = raw_directorandactor.replace("\n","")
        
            #Separate directors and actors
        director = directorandactor.split('|')[0]
        actor = directorandactor.split('|')[1]
            
            #Data preprocessing 
                #Director
        director_no_space = director.replace('\n','')
        director_final = director_no_space.replace('Director:','')
        director_final = director_final.replace('Directors:','') 
        directors.append(director_final)
                #Actor
        actor_no_space = actor.replace('\n','')
        actor_final = actor.replace('Stars:', '')
        actors.append(actor_final)
        


In [19]:
import pandas as pd

movies_df = pd.DataFrame({'rank': ranks,
                        'movie': titles,
                        'description': descriptions,
                        'runtime': runtimes,
                        'genre': genres,
                        'rating': ratings,
                        'metascore': metascores,
                        'votes': votes,
                        'gross earning in millions': gross_earnings,
                        'directors': directors,
                        'actors': actors
                       })
print(movies_df.info())
movies_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 11 columns):
actors                       50 non-null object
description                  50 non-null object
directors                    50 non-null object
genre                        50 non-null object
gross earning in millions    50 non-null float64
metascore                    50 non-null int64
movie                        50 non-null object
rank                         50 non-null object
rating                       50 non-null float64
runtime                      50 non-null int64
votes                        50 non-null int64
dtypes: float64(2), int64(3), object(6)
memory usage: 4.4+ KB
None


,actors,description,directors,genre,gross earning in millions,metascore,movie,rank,rating,runtime,votes
0,"Owen Wilson, Vince Vaughn, Rachel McAdams...","John Beckwith and Jeremy Grey, a pair of c...",David Dobkin,"Comedy, Romance",209.22,64,Wedding Crashers,1.,7.0,119,311714
1,"Will Ferrell, Christina Applegate, Steve ...",Ron Burgundy is San Diego's top-rated news...,Adam McKay,Comedy,85.29,63,Anchorman: The Legend of Ron Burgundy,2.,7.2,94,305428
2,"John C. Reilly, Jenna Fischer, David Krum...",Singer Dewey Cox overcomes adversity to be...,Jake Kasdan,"Comedy, Music",18.32,63,Walk Hard: The Dewey Cox Story,3.,6.8,96,62217
3,"Will Ferrell, John C. Reilly, Mary Steenb...",Two aimless middle-aged losers still livin...,Adam McKay,Comedy,100.47,51,Step Brothers,4.,6.9,98,244019
4,"Zach Galifianakis, Bradley Cooper, Justin...",Three buddies wake up from a bachelor part...,Todd Phillips,Comedy,277.32,73,The Hangover,5.,7.7,100,662818
5,"Ron Livingston, Jennifer Aniston, David H...",Three company workers who hate their jobs ...,Mike Judge,Comedy,10.82,68,Office Space,6.,7.8,89,224752
6,"Jim Carrey, Jeff Daniels, Lauren Holly, M...",The cross-country adventures of 2 good-hea...,"Peter Farrelly, Bobby Farrelly",Comedy,127.18,41,Dumb and Dumber,7.,7.3,107,326117
7,"Mike Myers, Elizabeth Hurley, Michael Yor...",A 1960s secret agent is brought out of cry...,Jay Roach,"Adventure, Comedy",53.88,51,Austin Powers: International Man of Mystery,8.,7.0,89,204639
8,"Mike Myers, Heather Graham, Michael York,...",Dr. Evil is back and has invented a new ti...,Jay Roach,"Action, Adventure, Comedy",206.04,59,Austin Powers: The Spy Who Shagged Me,9.,6.6,95,201854
9,"Ben Stiller, Owen Wilson, Snoop Dogg, Vin...",Two streetwise cops bust criminals in thei...,Todd Phillips,"Comedy, Crime",88.24,55,Starsky & Hutch,10.,6.1,101,130632
